In [ ]:
# Welcome to my first summarization chat bot

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [5]:
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [24]:
ed = Website("https://cnn.com")
print(ed.title)
print(ed.text)

Breaking News, Latest News and Videos | CNN
CNN values your feedback
1. How relevant is this ad to you?
2. Did you encounter any technical issues?
Video player was slow to load content
Video content never loaded
Ad froze or did not finish loading
Video content did not start after ad
Audio on ad was too loud
Other issues
Ad never loaded
Ad prevented/slowed the page from loading
Content moved around while ad loaded
Ad was repetitive to ads I've seen previously
Other issues
Cancel
Submit
Thank You!
Your effort and contribution in providing this feedback is much
                                        appreciated.
Close
Ad Feedback
Close icon
US
World
Politics
Business
Health
Entertainment
Underscored
Style
Travel
Sports
Science
Climate
Weather
Ukraine-Russia War
Israel-Hamas War
Games
More
US
World
Politics
Business
Health
Entertainment
Underscored
Style
Travel
Sports
Science
Climate
Weather
Ukraine-Russia War
Israel-Hamas War
Games
Watch
Listen
Live TV
Subscribe
Sign in
My Account
Settin

In [25]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown. When you give response to user, please use some emoji to make the response fun!"

In [26]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [27]:
print(user_prompt_for(ed))

You are looking at a website titled Breaking News, Latest News and Videos | CNN
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

CNN values your feedback
1. How relevant is this ad to you?
2. Did you encounter any technical issues?
Video player was slow to load content
Video content never loaded
Ad froze or did not finish loading
Video content did not start after ad
Audio on ad was too loud
Other issues
Ad never loaded
Ad prevented/slowed the page from loading
Content moved around while ad loaded
Ad was repetitive to ads I've seen previously
Other issues
Cancel
Submit
Thank You!
Your effort and contribution in providing this feedback is much
                                        appreciated.
Close
Ad Feedback
Close icon
US
World
Politics
Business
Health
Entertainment
Underscored
Style
Travel
Sports
Science
Climate
Weather
Ukraine-Russia War
Israel-Hamas War
Games
Mo

In [28]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [29]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown. When you give response to user, please use some emoji to make the response fun!'},
 {'role': 'user',
  'content': "You are looking at a website titled Breaking News, Latest News and Videos | CNN\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nCNN values your feedback\n1. How relevant is this ad to you?\n2. Did you encounter any technical issues?\nVideo player was slow to load content\nVideo content never loaded\nAd froze or did not finish loading\nVideo content did not start after ad\nAudio on ad was too loud\nOther issues\nAd never loaded\nAd prevented/slowed the page from loading\nContent moved around while ad loaded\nAd was repetitive to ads I've seen previously\nOther i

In [30]:
def summarize(url):
    website = Website(url)
    payload = {
        "model": MODEL,
        "messages": messages_for(ed),
        "stream": False
    }
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    return response.json()['message']['content']

In [31]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [32]:
summarize("https://edwarddonner.com")

'The latest news and updates from around the world.\n\n**Politics**\n\n* Donald Trump has asked the Supreme Court to let him fire Lisa Cook, a Federal Reserve official, over her comments on inflation.\n* Erika Kirk has been named CEO of Turning Point USA, a conservative organization.\n\n**Business**\n\n* The FTC is suing Live Nation and Ticketmaster over "illegal ticket resale" practices.\n* Hyundai\'s CEO found out about an ICE raid at a Georgia battery plant on the news.\n\n**Technology**\n\n* Apple has released its new iPhone Air, which is said to be the thinnest phone ever made by the company.\n\n**Entertainment**\n\n* Bad Bunny, Amy Poehler, and Sabrina Carpenter are among the first three hosts of \'SNL\' Season 51.\n* Cardi B reveals she\'s expecting a child with NFL player Stefon Diggs.\n\n**Health**\n\n* A study has found that chimps may ingest equivalent to 1 alcoholic drink a day from fermented fruit.\n* The CDC recommends COVID-19 vaccines for everyone in the US to save thou

In [33]:
import ollama

response = ollama.chat(model=MODEL, messages=messages_for(ed))
print(response['message']['content'])

Here are the top news stories from around the world:

**US**

* Pete Buttigieg rallies Indiana Democrats against GOP push to draw new congressional maps
* Recommending Covid vaccines for everyone in the US could save thousands more lives than limiting to high-risk groups
* How the attack-dog FCC chair helped take down Kimmel with words, not actions

**Politics**

* Trump asks Supreme Court to let him fire Lisa Cook from Federal Reserve
* Erika Kirk named CEO of Turning Point USA
* What is Antifa, the leftist movement Trump says he’s labeling ‘major terrorist organization’?

**Business**

* Hyundai CEO found out about ICE raid at Georgia battery plant on the news
* The FTC is suing Live Nation and Ticketmaster over ‘illegal ticket resale’ practices

**World**

* Hamas warns Gaza City offensive ends hope for hostage return, as pressure from families on Netanyahu grows
* UK intelligence service MI6 launches dark web portal called ‘Silent Courier’ to recruit foreign spies
* Sally Rooney sa

In [34]:
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages_for(ed)
)

print(response.choices[0].message.content)

Here's a summary of the top news stories from around the world:

**Politics and Business**

* President Trump asks the Supreme Court to let him fire Lisa Cook from the Federal Reserve (USA)
* The FTC sues Live Nation and Ticketmaster over illegal ticket resale practices (USA)
* Hyundai CEO discovers ICE raid at Georgia battery plant on news (South Korea)

**Environment and Climate**

* A study says that cows painted like zebras and pizza-eating lizards are a part of the Ig Nobel Prizes awarded for their absurdity (Science)
* The Federal Reserve fights a "two-front war" to combat inflation and economic uncertainty (USA)

**Technology and Entertainment**

* Apple tests its new iPhone Air, the thinnest phone ever made (Technology)
* Bad Bunny, Amy Poehler, and Sabrina Carpenter are announced as the first hosts of SNL Season 51 (Entertainment)

**Sports and Health**

* The Atlanta Braves' ace Clayton Kershaw announces his retirement from baseball (Sports)
* Study: Chimps may ingest equival